In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [17]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

        except:
            price = ""

    return price


# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-3-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Number of User Reviews
def get_delivery(soup):
    try:
        delivery = soup.find("span", attrs={"class":'a-text-bold'}).string.strip()

    except AttributeError:
        delivery = ""

    return delivery

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [18]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=oneplus+phone&i=electronics&sprefix=oneplus+pho%2Celectronics%2C278&ref=nb_sb_ss_ts-doa-p_1_11"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[] ,"rating":[] ,"reviews":[], "delivery":[], "availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['delivery'].append(get_delivery(new_soup))
        d['availability'].append(get_availability(new_soup))
    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [19]:
amazon_df

,title,price,rating,reviews,delivery,availability
0,"Samsung Galaxy S20 FE 5G (Cloud Mint, 8GB RAM,...","₹37,990.00",4.2 out of 5 stars,"26,040 ratings",With Exchange,In stock.
1,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM...","₹18,999.00",4.3 out of 5 stars,"100,663 ratings",With Exchange,In stock.
2,"OnePlus Nord 2T 5G (Jade Fog, 8GB RAM, 128GB S...","₹28,999.00",4.3 out of 5 stars,"14,554 ratings",With Exchange,In stock.
3,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM,...","₹18,999.00",4.3 out of 5 stars,"100,663 ratings",With Exchange,In stock.
4,"OnePlus 10R 5G (Sierra Black, 12GB RAM, 256GB ...","₹38,999.00",4.2 out of 5 stars,"9,948 ratings",With Exchange,In stock.
5,"OnePlus 10 Pro 5G (Emerald Forest, 8GB RAM, 12...","₹61,999.00",4.2 out of 5 stars,"4,902 ratings",With Exchange,In stock.
6,"OnePlus Nord 2T 5G (Gray Shadow, 8GB RAM, 128G...","₹28,999.00",4.3 out of 5 stars,"14,554 ratings",With Exchange,In stock.
7,"OnePlus 10R 5G (Forest Green, 12GB RAM, 256GB ...","₹38,999.00",4.2 out of 5 stars,"9,948 ratings",With Exchange,In stock.
8,"OnePlus Nord 2T 5G (Jade Fog, 12GB RAM, 256GB ...","₹33,999.00",4.3 out of 5 stars,"14,554 ratings",With Exchange,In stock.
9,"OnePlus 10T 5G (Moonstone Black, 12GB RAM, 256...","₹54,999.00",4.3 out of 5 stars,"2,541 ratings",With Exchange,In stock.
